In [ ]:
#-------- Loading required packages ----------
from datetime import timedelta
from datetime import datetime as dt_obj
import drms
from IPython.display import clear_output
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import os
from shutil import copyfile
import tarfile
import time
import urllib
import urllib.parse as parse



In [ ]:
# ### =============== Download part ================

#==== Set up download parameters ===

p1 = 0         # start number from the request list
p2 = 26      # end number from the request list

email    = 'aditi.hmidatadownload@gmail.com'  # JSOC email list
datadir = '/MS/ab15ms120/ML/final_updated/requests/'  # where request and error files are saved
reqfile  = datadir + 'new_active_harp_doppler_requests_{0}_{1}.txt'.format(p1,p2)  #!!! contains the list of jsoc request id's
outdir      = '/home/ML/AR_download_again/' # folder to download the ar data 
method   = 'url-tar'

if not os.path.exists(outdir):
            os.mkdir(outdir)


# read the request file to dataframe. Update column names if needed
req= pd.read_csv(reqfile,sep='\t',header=None,names=['index','date', 'requestid', 'resp','size','wait_time'])
req['index'] = req['index'].apply(str) #split the i_j index into i and j
req[['i', 'j']] = req['index'].str.split('_',expand=True)
req = req.drop(['index'], axis=1)
cols=['i','j','date', 'requestid', 'resp','size','wait_time']
reqf = req.reindex(cols,axis=1)
reqf



reqf[reqf.duplicated('date',keep=False)]



reqf2=reqf.drop_duplicates(subset ="date", keep = 'first').reset_index(drop=True)
reqf2



ilist = reqf2['i']
jlist = reqf2['j']
requestlist = reqf2['requestid']
resplist    = reqf2['resp']
datelist = reqf2['date']

ival  = ilist.drop_duplicates().reset_index(drop=True)
#dates = datelist.drop_duplicates().reset_index(drop=True)
icount = len(ival)

# collect all data for a given noaa into a single dataframe
def index_data(inum):
    idf = reqf[reqf['i'] == str(inum)]
    return idf
icount

In [ ]:
#===== Function for downloading data from JSOC for a given request id ====
# inputs:
# date : date
# request_id: request id as generated before
# outdir: path for saving the downloaded files
c   = drms.Client(email=email, verbose=True) # setup the drms client
def jsoc_download(date,request_id,outdir, num):
    num = num 
    # create the individual folders for different index
    out_dir    = outdir + 'AR_'+ str(date)
    if not os.path.exists(out_dir):
            os.mkdir(out_dir)
    # create empty files for each request id to avoid repeat downloads
    export_file = out_dir+'/jsoc_export_'+ request_id
    if os.path.exists(export_file):
        print("Download already exists! Skipping!!!")
        time.sleep(5)
        return None

    # create the request object from id
    r=c.export_from_id(request_id)

    # check the download status
    check = r.status
    if check > 0:
        print('Status = ', check)
        print('\nDownload not ready. Try again later')
        return None

    # create a file containing details of all the downloaded files
    Path(out_dir+'files.txt').touch()
    r.data.to_csv(out_dir+'files.txt',header=True, index=False, sep='\t', mode='a')

    # download the files
    r.download(out_dir)

    # untar them if needed then delete the tar file
    if method == 'url-tar':
        tarf = out_dir+'/'+request_id+'.tar'
        outdir_tar = out_dir+'/'+request_id
        if not os.path.exists(outdir_tar):
            os.mkdir(outdir_tar)
        tf   = tarfile.open(tarf)
        tf.extractall(outdir_tar)
        tf.close()
    Path(export_file).touch()
    return None

# print the index numbers and the count the jsoc requests to download
print('i\tj\tdate\t')
for i in range(icount):
    inum   = ival[i]
    idf = index_data(inum)
    idate = idf.date.values[0]
#     itime = idf.time.values[0]
    fcount = len(idf)
    print('{0}\t{1}\t{2}'.format(inum,fcount,idate))


In [ ]:
# ====== Loop for the JSOC downloads ===========
#--------- TO BE CHANGED -------------

n1 = 0         # start number for index; = 0 if starting from begining
n2 = icount     # end number; = icount if doing all

# for loop for the download
for i in range(n1, n2):
    inum   = ival[i]
    idf = index_data(inum)
    idate = idf.date.values[0]
    ireq  = idf.requestid.values
    fcount = len(idf)
    
    m1 = 0         # start number for the segment; typically 0
    m2 = fcount    # end number; typically filecount, fcount


    for j in range(m1, m2):
        print('{0}\t{1}\t{2}/{3}'.format(i,idate, j+1, m2))
        num = str(j)
        request_id = ireq[j]
        now = dt_obj.now()
        print('now = ', now)
        jsoc_download(idate,request_id,outdir, num)
    #clear_output()